In [ ]:
import pandas as pd
import numpy as np
import nltk
import glob

In [ ]:
contents = []
overalls = []
for filename in glob.iglob('./Review_Texts/*.dat', ):
#for filename in glob.iglob('./test_hotel/*.dat', ):
    with open(filename, 'r', encoding="utf8") as input_file:
        lines = input_file.readlines()
        
        for line in lines:
            #print(line)
            if '<Content>' in line:
                contents.append(line[9:].strip('\n'))
            if '<Overall>' in line:
                overalls.append(line[9:].strip('\n'))
print(contents[:1])
print(overalls[:1])

In [15]:
df = pd.DataFrame(list(map(list, zip(contents, overalls))))
columns = ['review', 'score']
df.columns = columns

In [16]:
df.head()

,review,score
0,Wonderful time- even with the snow! What a gre...,5
1,"Lovely hotel, unique decor, friendly front des...",4
2,"Nice hotel, expensive parking We got a good de...",4
3,Fabulous hotel Location and service are great....,5
4,"Loved the Monaco! Staff was amazing, with a sm...",5


# 70-30 Test-Train split

In [20]:
X = df.review.values
y = df.score.values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4)

# TF-IDF

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_df = 0.6,
                             stop_words='english',
                             use_idf=True)
X_train = vectorizer.fit_transform(X_train)
print("  Actual number of tfidf features: %d" % X_train.get_shape()[1])

  Actual number of tfidf features: 180189


# Latent Semantic Analysis (dimensionality reduction)

In [22]:
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

svd = TruncatedSVD(100)
lsa = make_pipeline(svd, Normalizer(copy=False))

# Run SVD on the training data, then project the training data.
X_train_lsa = lsa.fit_transform(X_train)

explained_variance = svd.explained_variance_ratio_.sum()
print("  Explained variance of the SVD step: {}%".format(int(explained_variance * 100)))

  Explained variance of the SVD step: 13%


# Predicting score with Ordinary Least Squares Linear Regression

In [ ]:
from sklearn import linear_model

regr = linear_model.LinearRegression()

regr.fit(X_train, y_train)
print('Coefficients: \n', regr.coef_)
print("Mean squared error: %.2f"
      % np.mean((regr.predict(X_test) - y_test) ** 2))

print('Variance score: %.2f' % regr.score(X_test, y_test))
